<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_ChatGithub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install --upgrade langchain deeplake openai tiktoken

In [1]:
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.vectorstores import Chroma

os.environ['OPENAI_API_KEY'] = 'sk-'
os.environ['ACTIVELOOP_TOKEN'] = 't'


In [2]:
embeddings = OpenAIEmbeddings(disallowed_special=())

In [5]:
!git clone https://github.com/Tencent/Shadow.git

^C


Cloning into 'chroma'...
fatal: not a git repository: 'C:/Users/Dell/studio/python/auto-trade/examples/chatgpt/chroma/.git'
fatal: remote did not send all necessary objects


In [3]:
import os
from langchain.document_loaders import TextLoader

# root_dir = './tmp/shadow'
root_dir = 'C:\\Users\\Dell\\studio\\android\\Shadow'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try: 
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e: 
            pass

In [4]:
len(docs)

1659

In [5]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)


Created a chunk of size 1060, which is longer than the specified 1000
Created a chunk of size 1479, which is longer than the specified 1000
Created a chunk of size 1020, which is longer than the specified 1000
Created a chunk of size 1540, which is longer than the specified 1000
Created a chunk of size 1188, which is longer than the specified 1000
Created a chunk of size 1012, which is longer than the specified 1000
Created a chunk of size 1828, which is longer than the specified 1000
Created a chunk of size 1378, which is longer than the specified 1000
Created a chunk of size 1027, which is longer than the specified 1000
Created a chunk of size 1012, which is longer than the specified 1000
Created a chunk of size 1828, which is longer than the specified 1000
Created a chunk of size 1374, which is longer than the specified 1000
Created a chunk of size 2138, which is longer than the specified 1000
Created a chunk of size 1012, which is longer than the specified 1000
Created a chunk of s

In [6]:
len(texts)

3673

In [7]:
# username = "alphazz"
# db = DeepLake(dataset_path=f"hub://{username}/chroma_source2", embedding_function=embeddings, public=True)
# db.add_documents(texts)

vectordb = Chroma.from_documents(texts, embeddings, persist_directory="./tmp/shadow/data/chromadb")
vectordb.persist()

Using embedded DuckDB with persistence: data will be stored in: ./tmp/shadow/data/chromadb


In [11]:
# db = DeepLake(dataset_path=f"hub://{username}/chroma_source2", read_only=True, embedding_function=embeddings)

|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/alphazz/chroma_source2



|

hub://alphazz/chroma_source2 loaded successfully.

Deep Lake Dataset in hub://alphazz/chroma_source2 already exists, loading from the storage
Dataset(path='hub://alphazz/chroma_source2', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape       dtype  compression
  -------   -------    -------     -------  ------- 
 embedding  generic  (1074, 1536)  float32   None   
    ids      text     (1074, 1)      str     None   
 metadata    json     (1074, 1)      str     None   
   text      text     (1074, 1)      str     None   


In [8]:
db = vectordb
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 10


In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model='gpt-3.5-turbo')
qa = ConversationalRetrievalChain.from_llm(model,retriever=retriever)

In [10]:
questions = [
    "What does Shadow do?",
    "How to use Shadow?"
] 
chat_history = []

for question in questions:  
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"Question:\n {question} \n")
    print(f"Answer:\n {result['answer']} \n\n")

Question:
 What does Shadow do? 

Answer:
 Based on the context provided, it is not clear what Shadow is or does. The provided text is a comment section containing information about the license for a software product made available by Tencent. 


Question:
 How to use Shadow? 

Answer:
 Based on the provided context, Shadow is a software that is made available by Tencent to support the open-source community. It is licensed under the BSD 3-Clause License and can be used in compliance with the license terms. Unfortunately, the context does not provide specific information about the purpose or function of Shadow or how it can be utilized. 




In [11]:
def ask(question, chat_history):
  response = qa({"question": question, "chat_history": chat_history})
  print(f"Question:\n {question}\n")
  print(f"Answer:\n {response['answer']}\n")

In [12]:
ask("What's the main programming language used in Shadow?", chat_history)

Question:
 What's the main programming language used in Shadow?

Answer:
 The given context does not provide information on the main programming language used in Shadow.



In [13]:
ask('Summarize the storage part of Shadow', chat_history)

Question:
 Summarize the storage part of Shadow

Answer:
 I'm sorry, the provided context does not contain information specific to the storage part of Shadow. Can you please provide more context or clarify your question?



In [17]:
ask('Shadow 是什么', chat_history)

Question:
 Shadow 是什么

Answer:
 Based on the provided context, Shadow seems to be a software made available by Tencent to support the open source community. It is licensed under the BSD 3-Clause License. However, without more information it is unclear what the software actually does.



In [19]:
ask('Shadow 怎么使用，请用中文回答', chat_history)

Question:
 Shadow 怎么使用，请用中文回答

Answer:
 Shadow是一个腾讯开发的Android插件框架，可以让你在一个宿主应用中使用多个插件。Shadow主要有以下特点：

- 可以复用独立安装App的源码
- 零反射无Hack实现插件技术，从理论上就已经确定无需对任何系统做兼容开发，更无任何隐藏API调用
- 全动态插件框架，一次性实现完美的插件框架很难，但Shadow将这些实现全部动态化起来，使插件框架的代码成为了插件的一部分
- 宿主增量极小，得益于全动态实现，真正合入宿主程序的代码量极小（15KB，160方法数左右）

插件化开发的好处主要有两点：

- 系统分层隔离，提高稳定性。不同的模块通过插件化可以在不同的进程中，相互之间的影响大大降低，增加了系统的稳定性。
- 提高开发效率和管理效率。不同的模块维护在不同的代码仓库中，开发人员可以分别负责不同的模块，提高了开发效率，同时也方便了管理。



In [21]:
ask('Shadow 有哪几个 Samples App 示例，请用中文回答', chat_history)

Question:
 Shadow 有哪几个 Samples App 示例，请用中文回答

Answer:
 Shadow提供了以下两个示例工程：

- 源码依赖 SDK 的 Sample(`projects/sample/source`)
- 二进制 Maven 依赖 SDK 的 Sample(`projects/sample/maven`)



In [22]:
ask('Shadow 中 Samples DynamicAPK 示例是演示什么功能的，请用中文回答', chat_history)

Question:
 Shadow 中 Samples DynamicAPK 示例是演示什么功能的，请用中文回答

Answer:
 Shadow中Samples DynamicAPK示例演示了一个多插件环境，其中包含了一个宿主应用、一个插件管理器和多个插件。宿主应用打包了管理器，以及 Shadow 的核心实现，即 Loader 和 Runtime。管理器负责下载、安装和卸载插件。插件可以包含业务 App 和 Shadow 的核心实现，即 Loader 和 Runtime。其中业务 App 和 Loader、Runtime 是同一个版本的代码编译出的。因此，Loader 可以包含一些业务逻辑，针对业务进行特殊处理。由于 Loader 是多实例的，因此同一个宿主中可以有多种不同的 Loader 实现。在加载插件时，管理器需要先加载插件中的 Runtime 和 Loader，再通过 Loader 的 Binder 操作 Loader 进而加载业务 App。



In [23]:
ask('Shadow 中多插件 Samples 示例工程是哪个，请用中文回答', chat_history)

Question:
 Shadow 中多插件 Samples 示例工程是哪个，请用中文回答

Answer:
 Shadow中的多插件示例工程是在`projects/sample/source`目录中的`sample-app`和`sample-base`构成的。在Android Studio中可以直接运行`sample-host`模块来查看示例效果，同时阅读`sample-app`和`sample-base`的代码来了解多插件的实现方式。



In [24]:
ask('Shadow 中Classloader 插件加载怎么实现的，请用中文回答', chat_history)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002E817F0E370>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。')).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002E817F0EAF0>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。')).


Question:
 Shadow 中Classloader 插件加载怎么实现的，请用中文回答

Answer:
 在 Shadow 中，每个插件都有一个独立的 ClassLoader 来加载插件的类。这个 ClassLoader 是在插件的 ShadowRuntime 中创建的，它的父类加载器是宿主应用的 ClassLoader。这样可以保证插件类和宿主类在加载时相互独立，避免类冲突。同时，Shadow 还实现了插件间的类加载隔离，使得不同插件之间的类也是相互独立的。



In [25]:
ask('Shadow 中 Resource 资源加载怎么实现的，请用中文回答', chat_history)

Question:
 Shadow 中 Resource 资源加载怎么实现的，请用中文回答

Answer:
 在 Shadow 中，资源加载的实现是通过替换宿主程序的 Resources 对象，在插件的 Resources 对象中查找资源来实现的。具体实现方法如下：

1. 在插件代码中创建一个新的 Resources 对象，用于查找插件资源。代码示例：

   ```java
   AssetManager assetManager = createAssetManager(pluginApkFilePath);
   Resources pluginResources = new Resources(assetManager, hostResources.getDisplayMetrics(), hostResources.getConfiguration());
   ```

   在这个示例中，createAssetManager() 方法是用来创建 AssetManager 对象的，pluginApkFilePath 参数表示插件 apk 文件的路径。hostResources 参数表示宿主程序的 Resources 对象，这个参数用于设置插件资源的一些基本信息，例如屏幕密度、语言等。

2. 在宿主程序的 Context 中，替换掉 Resources 对象。代码示例：

   ```java
   Context appContext = context.getApplicationContext();
   try {
       Class<?> contextImplClass = Class.forName("android.app.ContextImpl");
       Field mResourcesField = contextImplClass.getDeclaredField("mResources");
       mResourcesField.setAccessible(true);
       mResourcesField.set(appContext, pluginResources);
   } catch (ClassNotFoundException | NoSuchFieldException | I